#DATAMINING PROJECT V1.7
***From Baptiste Danichert & Ahmed Abdel Aziz***

**Last update**: deleting Stopwords - 1.6

**Description:** You have noticed that to improve one’s skills in a new foreign language, it is important to read texts in that language. These texts have to be at the reader’s language level. However, it is difficult to find texts that are close to someone’s knowledge level (A1 to C2). You have decided to build a model for English speakers that predicts the difficulty of a French written text. This can be then used, e.g., in a recommendation system, to recommend texts, e.g, recent news articles that are appropriate for someone’s language level. If someone is at A1 French level, it is inappropriate to present a text at B2 level, as she won’t be able to understand it. Ideally, a text should have many known words and may have a few words that are unknown so that the person can improve.



# 1. Loading the training data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/training_data.csv')
df_pred = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/unlabelled_test_data.csv')

In [ ]:
df_pred.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


In [ ]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


# 2. Dataframe analysis


In [ ]:
df.shape

(4800, 3)

-> 4800 rows and 2 columns (excluding first column "id")

In [ ]:
df.isnull().sum()

id            0
sentence      0
difficulty    0
dtype: int64

-> no NAs

In [ ]:
df.duplicated(subset="sentence").value_counts()

False    4800
dtype: int64

-> no duplicate in the data

#3.  Baseline

In [ ]:
np.random.seed = 0

In [ ]:
base_rate = max(df.value_counts('difficulty'))/df.shape[0]
print('Base rate:', round(base_rate,4))

Base rate: 0.1694


# 4. Data Cleaning

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install -U wordstats
!python -m spacy download fr_core_news_sm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from wordstats import Word, common_words
from collections import Counter
import string
import nltk
import spacy
nlp = spacy.load("fr_core_news_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 5.6 MB/s 
     |████████████████████████████████| 1.2 MB 53.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wordstats: filename=wordstats-1.0.7-py3-none-any.whl size=3616363 sha256=c944d72d9d335f6a4c6e9ee521aaa651749653f5e3b0f3b9c64326701ae34837
  Stored in directory: /root/.cache/pip/wheels/07/27/7d/24ade697c516ed02f369a90c5fec463286a6a67a299f75b711
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34527 sha256=cb7f726f3fb92a6b9c5049f5e0ee14b4d69bca890f331df56a811c14f31c3efe
  Stored in directory: /root/.cache/pip/wheels/e0/ea/e1/7b17c0af6f0228de33d82556dbad4fe823d90696fc50f2c81d
Successfully built wordstats configobj
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWar

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
spacy_stopwords = spacy.lang.fr.stop_words.STOP_WORDS
print('Number of stopwords: %d' % len(spacy_stopwords))

Number of stopwords: 507


In [ ]:
def remove_stopwords(text):
    doc = nlp(text)
    words = [chunk.lemma_ for chunk in doc.noun_chunks if chunk.lemma_ not in nlp.Defaults.stop_words]
    return " ".join(words)
df['sentence'] = df['sentence'].apply(remove_stopwords)
df_pred['sentence'] = df_pred['sentence'].apply(remove_stopwords)

# 5. Classification Algorithms

Dependent variable (y) is the column named "difficulty".
<br>We split the data into 80% training and 20% test set.


In [ ]:
y = df['difficulty']
X = df['sentence']
X_pred = df_pred['sentence']

### i. Logistic Regression.
We use the following parameters for the LogisticRegressionCV():

* cross-validation to 5 folds
* maximum interation to 1000
* random state to 0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

classifier = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000, random_state=0)
pl = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier)])
pl.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier',
                 LogisticRegressionCV(cv=5, max_iter=1000, random_state=0))])

In [ ]:
def evaluate(true, pred):
    precision = precision_score(true, pred,average='macro')
    recall = recall_score(true, pred,average='macro')
    f1 = f1_score(true, pred,average='macro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
y_pred = pl.predict(X_test)
evaluate(y_test, y_pred)

CONFUSION MATRIX:
[[86 33 19  8 10  5]
 [58 45 30 11  8 12]
 [20 25 56 14 18 27]
 [10 16 17 53 23 25]
 [18  5 17 33 66 34]
 [12 11  8 13 34 80]]
ACCURACY SCORE:
0.4021
CLASSIFICATION REPORT:
	Precision: 0.3983
	Recall: 0.4024
	F1_Score: 0.3980


In [ ]:
predict = pl.predict(X_pred)

In [ ]:
submission= pd.DataFrame()
submission['id']= df_pred.index
submission['difficulty'] = predict

In [ ]:
submission.to_csv("submission.csv", index=False)

### ii. kNNeighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

grid = {'n_neighbors':np.arange(1,100),
        'p':np.arange(1,3),
        'weights':['uniform','distance']}

knn = KNeighborsClassifier()
classifier_knn = GridSearchCV(knn, grid, cv=5)

pl_knn = Pipeline([('vectorizer', tfidf),
                 ('classifier', classifier_knn)])

pl_knn.fit(X_train, y_train)

print("Hyperparameters:", classifier_knn.best_params_)

Hyperparameters: {'n_neighbors': 4, 'p': 1, 'weights': 'distance'}


In [ ]:
y_knn_predict = pl_knn.predict(X_test)
evaluate(y_test, y_knn_predict)

CONFUSION MATRIX:
[[101  48  12   0   0   0]
 [104  48  11   1   0   0]
 [118  28  11   2   0   1]
 [106  23  11   3   0   1]
 [133  22   8   2   6   2]
 [120  24   6   1   2   5]]
ACCURACY SCORE:
0.1812
CLASSIFICATION REPORT:
	Precision: 0.3704
	Recall: 0.1793
	F1_Score: 0.1291


In [ ]:
knn_predict = pl_knn.predict(X_pred)
submission_knn= pd.DataFrame()
submission_knn['id']= df_pred.index
submission_knn['difficulty'] = knn_predict
submission_knn.to_csv("submissionknn.csv", index=False)

###  iii. Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
pl_dtc = Pipeline([('vectorizer', tfidf),
                 ('classifier', dtc)])
pl_dtc.fit(X_train, y_train)
y_pred_dtc = pl_dtc.predict(X_test)

evaluate(y_test, y_pred_dtc)

CONFUSION MATRIX:
[[80 28 31  8  8  6]
 [57 40 32 15  7 13]
 [27 30 42 19 18 24]
 [11 16 23 39 25 30]
 [ 6 17 31 38 34 47]
 [15 17 25 33 32 36]]
ACCURACY SCORE:
0.2823
CLASSIFICATION REPORT:
	Precision: 0.2780
	Recall: 0.2831
	F1_Score: 0.2784


In [ ]:
dtc_predict = pl_dtc.predict(X_pred)
submission_dtc= pd.DataFrame()
submission_dtc['id']= df_pred.index
submission_dtc['difficulty'] = dtc_predict
submission_dtc.to_csv("submissiondtc.csv", index=False)

### iv. Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
pl_rfc = Pipeline([('vectorizer', tfidf),
                 ('classifier', rfc)])

pl_rfc.fit(X_train, y_train)

y_pred_rfc = pl_rfc.predict(X_test)

evaluate(y_test, y_pred_rfc)

CONFUSION MATRIX:
[[102  30  20   5   3   1]
 [ 76  52  25   8   1   2]
 [ 39  48  51  12   2   8]
 [ 21  14  22  47  15  25]
 [ 14  15  25  41  45  33]
 [ 20  18  15  16  17  72]]
ACCURACY SCORE:
0.3844
CLASSIFICATION REPORT:
	Precision: 0.4015
	Recall: 0.3853
	F1_Score: 0.3791


In [ ]:
rfc_predict = pl_rfc.predict(X_pred)
submission_rfc= pd.DataFrame()
submission_rfc['id']= df_pred.index
submission_rfc['difficulty'] = rfc_predict
submission_rfc.to_csv("submissionrfc.csv", index=False)

### v. Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()
pl_svc = Pipeline([('vectorizer', tfidf),
                 ('classifier', svc)])

pl_svc.fit(X_train, y_train)

y_pred_svc = pl_svc.predict(X_test)

evaluate(y_test, y_pred_svc)

CONFUSION MATRIX:
[[80 32 21 17  6  5]
 [52 46 27 18  9 12]
 [17 21 48 27 19 28]
 [ 3  8 16 65 18 34]
 [ 6  3 17 39 65 43]
 [ 6 10  3 27 33 79]]
ACCURACY SCORE:
0.3990
CLASSIFICATION REPORT:
	Precision: 0.3997
	Recall: 0.4007
	F1_Score: 0.3956


In [ ]:
svc_predict = pl_svc.predict(X_pred)
submission_svc= pd.DataFrame()
submission_svc['id']= df_pred.index
submission_svc['difficulty'] = svc_predict
submission_svc.to_csv("submissionsvc.csv", index=False)

###vi. AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
pl_abc = Pipeline([('vectorizer', tfidf),
                 ('classifier', abc)])

pl_abc.fit(X_train, y_train)

y_pred_abc = pl_abc.predict(X_test)

evaluate(y_test, y_pred_abc)

CONFUSION MATRIX:
[[84 12 28 29  2  6]
 [63 21 31 30  3 16]
 [33 16 25 47  8 31]
 [11  2 17 45 20 49]
 [13  4 12 48 28 68]
 [12  2 18 29 12 85]]
ACCURACY SCORE:
0.3000
CLASSIFICATION REPORT:
	Precision: 0.3104
	Recall: 0.3031
	F1_Score: 0.2814


In [ ]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier()
pl_mlpc = Pipeline([('vectorizer', tfidf),
                 ('classifier', mlpc)])

pl_mlpc.fit(X_train, y_train)

y_pred_mlpc = pl_mlpc.predict(X_test)

evaluate(y_test, y_pred_mlpc)

CONFUSION MATRIX:
[[94 28 24  8  5  2]
 [65 55 23  9  5  7]
 [28 43 49 15 12 13]
 [12 23 22 50 19 18]
 [14 16 22 35 61 25]
 [11 22 21 21 20 63]]
ACCURACY SCORE:
0.3875
CLASSIFICATION REPORT:
	Precision: 0.3954
	Recall: 0.3873
	F1_Score: 0.3860


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mlpc_predict = pl_mlpc.predict(X_pred)
submission_mlpc= pd.DataFrame()
submission_mlpc['id']= df_pred.index
submission_mlpc['difficulty'] = mlpc_predict
submission_mlpc.to_csv("submissionmlpc.csv", index=False)

# 6. Comparing the models

In [ ]:
import pandas as pd

df_comparison = pd.DataFrame(columns=['Model_Name', 'Base_Rate', 'Precisions','Recall','F1-Score', 'Accuracy'])


df_comparison['Model_Name'] = ['Logistic Reg', 'KNN', 'Tree', 'Random Forest', 'Support Vector', 'AdaBooster', 'MLPC']
df_comparison['Base_Rate'] = ['0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694']
df_comparison['Precisions'] = ['0.3983', '0.3704', '0.2780','0.4015', '0.3997', '0.3104', '0.3954']
df_comparison['Recall'] = ['0.4024', '0.1793', '0.2831','0.3853', '0.4007', '0.3031', '0.3873']
df_comparison['F1-Score'] = ['0.3980', '0.1291', '0.2784','0.3791', '0.3956', '0.2814', '0.3860']
df_comparison['Accuracy'] = ['0.4021', '0.1812', '0.2823','0.3844', '0.3990', '0.3000', '0.3875']


df_comparison